In [ ]:
!pip install pyarrow duckdb pandas numpy -q

# 📦 Instalação de Pacotes

Antes de começar, vamos instalar os pacotes necessários:

# Capitulo 09 Compute Functions

Notebook gerado automaticamente a partir do código fonte python.


## 📚 Importação de Bibliotecas

Importando as bibliotecas necessárias para o capítulo:

In [ ]:
import pyarrow as pa
import pyarrow.compute as pc
import duckdb
import numpy as np
import pandas as pd

## 🔧 Configuração Inicial

Preparando dados de exemplo e conexão com DuckDB:

In [ ]:
# Dados de exemplo globais
try:
    print("\nGerando dados de exemplo...")
    data = pa.table({
        'id': range(1000),
        'valor': np.random.randn(1000),
        'categoria': np.random.choice(['A', 'B', 'C'], 1000)
    })
    print(f"Tabela PyArrow criada: {data.num_rows} linhas")
except Exception as e:
    print(f"Erro ao criar dados: {e}")

# Conexão DuckDB
con = duckdb.connect()

## 1️⃣ Operações Vetorizadas

Operações vetorizadas de alta performance com Arrow:

In [ ]:
print(f"\n--- {'Operações vetorizadas'.upper()} ---")

# Exemplo com PyArrow Compute
valor_dobrado = pc.multiply(data['valor'], 2)
data_calculada = data.append_column('valor_dobrado', valor_dobrado)

print("Estatísticas básicas do PyArrow:")
print(f"Soma: {pc.sum(data['valor'])}")
print(f"Média: {pc.mean(data['valor'])}")

# Exemplo com DuckDB (vetorizado internamente)
resultado_duck = con.execute("""
    SELECT 
        AVG(valor) as media,
        SUM(valor) as total
    FROM data
""").fetch_arrow_table()

print("\nResultado via DuckDB (Arrow):")
print(resultado_duck.to_pandas())

## 2️⃣ String Operations

Manipulação eficiente de strings:

In [ ]:
print(f"\n--- {'String operations'.upper()} ---")

# Criando dados de string
nomes = pa.array(["João Silva", "Maria Santos", "Pedro Oliveira", "Ana Costa", None])
tabela_strings = pa.table({'nome': nomes})

# Operações de String com PyArrow Compute
maiusculas = pc.utf8_upper(tabela_strings['nome'])
comprimento = pc.utf8_length(tabela_strings['nome'])

# Extração de substring (ex: primeiro nome)
# Encontrar o primeiro espaço
espaco_index = pc.find_substring(tabela_strings['nome'], " ")
# Esta operação é mais complexa em Arrow puro sem kernels específicos, 
# mas DuckDB brilha em strings complexas.

# Exemplo com DuckDB para strings complexas
resultado_strings = con.execute("""
    SELECT 
        nome,
        upper(nome) as NOME_MAIUSCULO,
        split_part(nome, ' ', 1) as primeiro_nome,
        length(nome) as tamanho
    FROM tabela_strings
    WHERE nome IS NOT NULL
""").fetch_arrow_table()

print("Processamento de strings via DuckDB:")
print(resultado_strings.to_pandas())

## 3️⃣ Filtros e Máscaras

Aplicando filtros e máscaras para seleção de dados:

In [ ]:
print(f"\n--- {'Filtros e máscaras'.upper()} ---")

# Filtro via PyArrow Compute (Máscara Booleana)
mask = pc.greater(data['valor'], 1.0)
data_filtrada_arrow = data.filter(mask)

print(f"Linhas originais: {data.num_rows}")
print(f"Linhas com valor > 1.0 (Arrow): {data_filtrada_arrow.num_rows}")

# Filtro via DuckDB (SQL Pushdown)
# O DuckDB filtra o Arrow Table sem carregar tudo para memória SQL tradicional
data_filtrada_duck = con.execute("""
    SELECT * FROM data 
    WHERE valor > 1.0 AND categoria = 'A'
""").fetch_arrow_table()

print(f"Linhas com valor > 1.0 e categoria 'A' (DuckDB): {data_filtrada_duck.num_rows}")

# Exemplo de 'slice' (offset/limit)
data_slice = data.slice(0, 5)
print("\nPrimeiras 5 linhas (Slice):")
print(data_slice.to_pandas())

## 4️⃣ Agregações

Funções de agregação de dados:

In [ ]:
print(f"\n--- {'Agregações'.upper()} ---")

# Agregação por Grupo via DuckDB (mais simples que Arrow puro para GroupBy)
agregado_duck = con.execute("""
    SELECT 
        categoria,
        COUNT(*) as contagem,
        AVG(valor) as media_valor,
        MIN(valor) as min_valor,
        MAX(valor) as max_valor
    FROM data
    GROUP BY categoria
    ORDER BY categoria
""").fetch_arrow_table()

print("Agregações por categoria (DuckDB over Arrow):")
print(agregado_duck.to_pandas())

# Agregação simples via PyArrow Compute
print("\nAgregações globais (PyArrow Compute):")
print(f"Valor total: {pc.sum(data['valor'])}")

# Para strings, podemos usar value_counts para ver a frequência
counts = pc.value_counts(data['categoria'])
print("\nFrequência por categoria (Arrow):")
print(counts.to_pandas())

## 5️⃣ Joins

Operações de join entre tabelas:

In [ ]:
print(f"\n--- {'Joins'.upper()} ---")

# Criando tabela de metadados para Join
metadata = pa.table({
    'categoria': ['A', 'B', 'C', 'D'],
    'descricao': ['Alta Performance', 'Baixo Risco', 'Crítico', 'Desconhecido'],
    'prioridade': [1, 3, 2, 4]
})

# Join usando DuckDB (mais intuitivo)
join_duck = con.execute("""
    SELECT 
        d.id,
        d.valor,
        d.categoria,
        m.descricao,
        m.prioridade
    FROM data d
    LEFT JOIN metadata m ON d.categoria = m.categoria
    LIMIT 10
""").fetch_arrow_table()

print("Resultado do Join (DuckDB over Arrow):")
print(join_duck.to_pandas())

print("\n" + "="*60)
print(f"Fim do Capítulo 09")
print("="*60)